In [ ]:
!pip install geopandas metpy xarray[complete] -q
!pip install eccodes cfgrib rioxarray ecmwf.opendata 

In [ ]:
import xarray as xr
import cfgrib

# import cartopy.crs as ccrs
# import matplotlib.pyplot as plt
# from cartopy.io.shapereader import Reader
# import cartopy.io.shapereader as shpreader

# from cartopy.feature import ShapelyFeature
# import cartopy.feature as cf
import os 
from ecmwf.opendata import Client
# plt.style.available
#import colormaps as cmaps 
import concurrent
import numpy as np

# import cartopy.crs as crs
# import cartopy.feature as cfeature
# import geopandas as gpd


import numpy as np

# Libraries to assist with animation and visualisations
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()
client = Client(source="ecmwf")

import metpy.calc as mpcalc
from metpy.units import units

import json
import pandas as pd
from datetime import datetime, timedelta

import rioxarray

import xarray as xr
import cfgrib
# import eccodes
# import geopandas as gpd
# from shapely.geometry import Point
import json
import zipfile
import os
from os import walk



#file='dist.shp'
#gdf = gpd.read_file(file)
#gdf = gdf.to_crs("EPSG:4326")
#geometry_column = gdf['geometry']

# out="dist.shp"
# gdf.geometry[:].to_file(out)
# gdf.to_file("dist.geojson", driver='GeoJSON')

# od=ShapelyFeature(Reader('IND.shp').geometries(), ccrs.PlateCarree(),
#                                 linewidth = 1,facecolor='none', edgecolor=[0,0,0], alpha=0.7, )

In [ ]:

def dict_l(data,fm):
  data_dict=data.sel(longitude=slice(60, 100), latitude=slice(38, 0)).squeeze().to_dict(data='list', encoding=False)
  def convert_types(obj):
    if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
        return obj.isoformat()  # Convert datetime objects to ISO format
    elif isinstance(obj, timedelta):
        return obj.total_seconds()  # Convert timedelta to total seconds
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert numpy floats to native floats
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert numpy integers to native ints
    raise TypeError(f"Type {type(obj)} not serializable")

  file=f"{fm}.json"
  # Save the dictionary as a JSON file with the extended convert_types function
  with open(file, 'w') as json_file:
      json.dump(data_dict, json_file, indent=4, default=convert_types)

In [ ]:
# HRES:

# fc: Forecast.
# ENS:

# cf: Control forecast.
# pf: Perturbed forecast.
# em: Ensemble mean.
# es: Ensemble standard deviation.
# ep: Probabilities.
# Valid values for stream are:

# oper: Atmospheric fields from HRES - 00 UTC and 12 UTC.
# wave: Ocean wave fields from HRES - 00 UTC and 12 UTC.
# enfo: Atmospheric fields from ENS.
# waef: Ocean wave fields from ENS.
# scda: Atmospheric fields from HRES - 06 UTC and 18 UTC.
# scwv: Ocean wave fields from HRES - 06 UTC and 18 UTC.

In [ ]:
step = list(range(3, 145, 3))

client.retrieve(
    time=12,
    type="fc",
    stream='oper',step=step,
    param=['d','gh',"r","vo"],
    levelist=[ 850, 500],
    target="atms.grib2",
)

In [ ]:
ds_850=xr.open_dataset('atms.grib2',engine='cfgrib').sel(longitude=slice(60,100),latitude=slice(40,0),isobaricInhPa=850)
ds_500=xr.open_dataset('atms.grib2',engine='cfgrib').sel(longitude=slice(60,100),latitude=slice(40,0),isobaricInhPa=500)


In [ ]:
gh8=ds_850['gh']
d8=ds_850['d']
r8=ds_850['r']
vo8=ds_850['vo']

dict_l(gh8,"gh850")
dict_l(d8*1e5,"div850")
dict_l(r8,"rh850")
dict_l(vo8*1e5,"vo850")




gh5=ds_500['gh']
d5=ds_500['d']
r5=ds_500['r']
vo5=ds_500['vo']

dict_l(gh5,"gh500")
dict_l(d5*1e5,"div500")
dict_l(r5,"rh500")
dict_l(vo5*1e5,"vo500")

In [ ]:
step = list(range(3, 145, 3))


client.retrieve(
    time=12,
    type="fc",
    stream='oper',step=step,
    param=['tp','tcwv',"msl","2t"],
    target="atms.grib2",
)




In [ ]:
# import cfgrib
t2 = xr.open_dataset('atms.grib2',engine='cfgrib',decode_coords="all",
                     backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 't2m',}})['t2m']
ds=xr.open_dataset('atms.grib2',engine='cfgrib')
# ds1=ds.sel(longitude=slice(60,100),latitude=slice(40,0))
rf = ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tp'][:, :, :]  # Select a specific time slice
p =ds.sel(longitude=slice(60,100),latitude=slice(40,0))['msl'][:, :, :]/100  # Select a specific time slice
tcw=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tcwv']
# u10=xr.open_dataset('atms.grib2',engine='cfgrib',backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 'u10',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['u10']
# v10=xr.open_dataset('atms.grib2',engine='cfgrib',backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 'v10',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['v10']

w10=(u10**2+v10**2)**0.5
from metpy.units import units
rf2=rf* units('m').to(units.mm)
rf1=rf2.diff(dim='step').dropna(dim='step')

In [ ]:

def dict_l(data,fm):
  data_dict=data.sel(longitude=slice(60, 100), latitude=slice(38, 0)).squeeze().to_dict(data='list', encoding=False)
  def convert_types(obj):
    if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
        return obj.isoformat()  # Convert datetime objects to ISO format
    elif isinstance(obj, timedelta):
        return obj.total_seconds()  # Convert timedelta to total seconds
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert numpy floats to native floats
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert numpy integers to native ints
    raise TypeError(f"Type {type(obj)} not serializable")

  file=f"{fm}.json"
  # Save the dictionary as a JSON file with the extended convert_types function
  with open(file, 'w') as json_file:
      json.dump(data_dict, json_file, indent=4, default=convert_types)

In [ ]:
dict_l(t2,"2m_temp")
dict_l(rf1,"rf")
dict_l(p,"msl")
dict_l(tcw,"tcwv")
# dict(w10,"w10")
# dict(tcw,"tcwv")
